In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load anime dataset
df = pd.read_csv('A16_anime.csv')
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [3]:
# Dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [4]:
# Check missing values
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [18]:
# Fill missing genres with 'Unknown'
df['genre'] = df['genre'].fillna('Unknown')
# Fill missing rating values with mean
df['rating'] = df['rating'].fillna(df['rating'].mean())
# Fill missing type values
df['type'] = df['type'].fillna('Unknown')
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

#To convert text data into numerical form
genre_tfidf = tfidf.fit_transform(df['genre'])
genre_tfidf.shape


(12294, 47)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity
CS = cosine_similarity(genre_tfidf)
CS.shape


(12294, 12294)

In [23]:
#Content-based recommendation system.
def recommend_anime(anime_name, top_n):
    # Find the index of the given anime
    anime_idx = df[df['name'] == anime_name].index[0]
    # Get similarity scores for that anime
    sim_scores= CS[anime_idx]
    # Convert scores into a DataFrame
    sim_df = pd.DataFrame({
        'index': range(len(sim_scores)),
        'score': sim_scores,
        'rating': df['rating']
    })
    # Sort by similarity score and rating
    sim_df = sim_df.sort_values(by=['score', 'rating'], ascending=False) 
    # Remove the same anime itself
    sim_df = sim_df.iloc[1:top_n+1]
    # Return recommended anime names
    return df[['name', 'rating']].iloc[sim_df['index']]

In [25]:
recommend_anime("Gintama°", top_n=5)

,name,rating
4,Gintama&#039;,9.16
9,Gintama&#039;: Enchousen,9.11
8,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,9.10
12,Gintama,9.04
63,Gintama: Yorinuki Gintama-san on Theater 2D,8.60
